In [95]:
import mysql.connector
from sqlalchemy import create_engine
import csv
import pandas as pd

## Variables de conexion base de datos y sus métodos de conexión

In [96]:
host='localhost'
database='bdProyectoBD_215215639'
user='root'
port = '3306'
password = "Lapicero.5"
## Otras variables
rutaArchivoCSV = "./bd.csv"

In [111]:
def abrirConexionBD():
 
    try:
        conn = mysql.connector.connect(
            host=host,
            database= database,
            user= user,
            password=password)
   
        if conn.is_connected():
            db_Info = connection.get_server_info()
            return  conn
            
    except Error as e:
        print("Error al conectar a MySQL", e)    

In [98]:
def cerrarConexionBD():
 
    try:
        conn = mysql.connector.connect(
            host=host,
            database= database,
            user= user,
            password=password)
   
    
        if conn.is_connected():
            connection.close()

    except Error as e:
        print("Error al conectar a MySQL", e) 

In [99]:
def ejecutarConsulta(conexion):

    cursor = conexion.cursor()
    
    query = '''
                SELECT 
                    EH.id,
                    EH.claveUsuario,
                    EH.fechaRegistro,
                    EH.edadEgreso AS EdadEgreso,
                    EH.diasEstancia AS DiasEstancia,
                    GE.grupo_edad AS GrupoEdad,
                    IA.nombre AS Istitucion,
                    IT.intervalo_tiempo AS Intervalo,
                    ME.motivo AS MotivoEgreso,
                    S.sexo AS Sexo,
                    UTT.unidad_tiempo AS UnidadTiempo
                FROM
                EgresosaHospital AS EH
                    INNER JOIN
                GruposEdades AS GE ON EH.idGruposEdades = GE.id
                    INNER JOIN
                Instituciones AS IA ON EH.idInstitucion = IA.id
                    INNER JOIN
                IntervalosTiempos AS IT ON EH.idIntervaloTiempos = IT.indice
                    INNER JOIN
                MotivosEgreso AS ME ON EH.idMotivoEgreso = ME.id
                    INNER JOIN
                Sexos AS S ON EH.idSexo = S.id
                    INNER JOIN
                UnidadesTiempo AS UTT ON EH.idUnidadesTiempo = UTT.id
    '''
    
    cursor.execute(query)
    
    resultado = cursor.fetchall()
    print(resultado)

    cursor.close()

In [100]:
df = pd.read_csv(rutaArchivoCSV)
df.head()

,claveUsuario,fechaRegistro,edadEgreso,diasEstancia,idInstitucion,IdMotivoEgreso,idIntervaloTiempos,idSexo,idGruposEdades,idUnidadesTiempo
0,2500041,01/05/22,29.0,2.0,4.0,2.0,20.0,2.0,4.0,5.0
1,2500047,01/06/22,1.0,2.0,4.0,7.0,20.0,1.0,2.0,5.0
2,2500052,01/04/22,46.0,5.0,4.0,2.0,50.0,1.0,5.0,5.0
3,2500155,01/06/22,56.0,3.0,4.0,2.0,30.0,1.0,5.0,5.0
4,2500157,01/06/22,46.0,2.0,4.0,2.0,20.0,1.0,5.0,5.0


In [120]:
def insertarRegistrosDF(conn):

    cursor = conn.cursor()

    for index, row in df.iterrows():

            sql = ''' 
                INSERT INTO EgresosaHospital
                    (
                        claveUsuario, fechaRegistro, edadEgreso, diasEstancia, 
                        idInstitucion, idMotivoEgreso, idIntervaloTiempos,
                        idSexo, idGruposEdades, idUnidadesTiempo
                     ) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
             '''
            cursor.execute(sql, tuple(row))
            
            conn.commit()


In [121]:
conn = abrirConexionBD()
insertarRegistrosDF(conn)

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`bdproyectobd_215215639`.`egresosahospital`, CONSTRAINT `egresosahospital_ibfk_3` FOREIGN KEY (`idIntervaloTiempos`) REFERENCES `intervalostiempos` (`id`))